In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import doubleml as dml
from tqdm.notebook import tqdm

In [3]:
X = pd.read_csv("C:/_Dissertation/Datasets/X_nt_normalised.csv", index_col=0)

In [4]:
%%time
for knob in range(9):
    knob_RF_results = pd.DataFrame(index=range(10), columns=["true_ate", "summary", "all_coef", "all_se"])
    for dataset in tqdm(range(10)):
        df = pd.read_csv("C:/_Dissertation/Datasets/knob_"+str(knob)+"/dataset_"+str(dataset)+".csv", index_col=0)
        ite = df['ite']
        y = df['y']
        D = df['D']
        data = X.copy()
        data['y'] = y
        data['D'] = D
        dml_data = dml.DoubleMLData(data=data, y_col='y', d_cols='D')
        g_estimator = RandomForestRegressor()
        m_estimator = RandomForestClassifier()
        dml_model = dml.DoubleMLIRM(dml_data, g_estimator, m_estimator, n_folds=5, n_rep=10, trimming_threshold=0.01)
        dml_model.fit(n_jobs_cv=3, store_predictions=True)
        knob_RF_results.loc[dataset] = ite.mean(), dml_model.summary, dml_model.all_coef, dml_model.all_se
        y_0_hat = dml_model.predictions['ml_g0'][:, :, 0].mean(axis=1)
        y_1_hat = dml_model.predictions['ml_g1'][:, :, 0].mean(axis=1)
        p_hat = dml_model.predictions['ml_m'][:, :, 0].mean(axis=1)
        preds = pd.DataFrame(columns=['y0hat', 'y1hat', 'phat'])
        preds['y0hat'] = y_0_hat
        preds['y1hat'] = y_1_hat
        preds['phat'] = p_hat
        preds.to_csv("C:/_Dissertation/Results/DML RF new/knob_"+str(knob)+"_dataset_"+str(dataset)+".csv")
    results = pd.DataFrame(columns=range(20))
    results['true_ate'] = knob_RF_results['true_ate']
    for column in knob_RF_results['summary'].loc[0].columns:
        results[column] = [x[0] for x in [df[column].values for df in knob_RF_results['summary']]]
    for estimate in range(10):
        for dataset in range(10):
            results[estimate].loc[dataset] = knob_RF_results['all_coef'].loc[dataset][0][estimate]
            results[estimate+10].loc[dataset] = knob_RF_results['all_se'].loc[dataset][0][estimate]
    results.to_csv("C:/_Dissertation/Results/DML RF new/knob_"+str(knob)+"_ATE.csv")



Wall time: 3h 8min 19s


C:\Programming\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
